# Baseline v3 - Unsloth 최적화 버전

이 베이스라인 코드는 기존 v3 코드를 **Unsloth 라이브러리**를 사용하여 최적화한 버전입니다.

**주요 변경 사항:**
- **Unsloth 적용**: 더 적은 메모리로 2배 이상 빠른 파인튜닝을 수행합니다.
- **SFTTrainer 사용**: Transformers의 복잡한 수동 학습 루프를 `trl`의 `SFTTrainer`로 대체하여 코드를 간소화했습니다.
- **`FastVisionModel` 사용**: Unsloth의 Vision-Language 모델 로딩 클래스를 사용하여 모델 로딩 및 양자화 과정을 단순화했습니다.

Colab의 GPU 환경(T4 GPU)에서 개발되었습니다.
- **런타임 > 런타임 유형 변경 > T4 GPU**로 설정해주세요.

> 참고 : https://docs.unsloth.ai/models/qwen3-vl-run-and-fine-tune  
> 참고 : https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_VL_(8B)-Vision-GRPO.ipynb#scrollTo=6fUaoYJEKgpb

# 1. 환경 준비

Unsloth 및 최신 라이브러리를 설치합니다.

- 아래 셀 실행 후 **런타임 > 세션 다시 시작**을 반드시 진행해주세요.

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2

# 2. 데이터 준비

구글 드라이브를 마운트하고 대회 데이터를 압축 해제합니다.   
(*kaggle로 진행하는 교육생들은 `/kaggle/input/`에 있는 데이터셋으로 진행해주세요!)

- `train.csv`, `train/`
- `test.csv`, `test/`
- `sample_submission.csv`

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 데이터 압축 해제 (약 1분 소요)
!unzip -q "/content/drive/My Drive/251024/data.zip" -d "/content/"

# 3. 라이브러리, 데이터, 설정 로드

In [ ]:
import os, random
import pandas as pd
from PIL import Image
import torch
from datasets import Dataset, DatasetDict
from unsloth import FastVisionModel
from trl import SFTTrainer, SFTConfig
from unsloth.chat_templates import get_chat_template
from unsloth.trainer import UnslothVisionDataCollator
from tqdm import tqdm

# 이미지 로드 시 픽셀 제한 해제
Image.MAX_IMAGE_PIXELS = None

# 디바이스 GPU 우선 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# 설정값 정의
MODEL_ID = "unsloth/Qwen3-VL-8B-Instruct-unsloth-bnb-4bit" # Unsloth의 4bit 양자화 모델
MAX_SEQ_LENGTH = 2048
MAX_NEW_TOKENS = 8
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 데이터셋 로드
train_df = pd.read_csv("/content/train.csv")
test_df  = pd.read_csv("/content/test.csv")

# 학습 시간을 줄이기 위해 200개 샘플만 사용 (실제 대회에서는 전체 데이터를 사용하세요)
train_df = train_df.sample(n=200, random_state=SEED).reset_index(drop=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Device: cuda


# 4. Unsloth 모델 및 LoRA 어댑터 로딩

Unsloth의 `FastVisionModel`을 사용하여 4bit 양자화된 모델과 토크나이저를 간편하게 로드합니다. 이후 LoRA 어댑터를 추가하여 학습 준비를 마칩니다.

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)

# LoRA 어댑터 추가
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,  # False if not finetuning vision layers
    finetune_language_layers   = True,  # False if not finetuning language layers
    finetune_attention_modules = True,  # False if not finetuning attention layers
    finetune_mlp_modules       = True,  # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = SEED,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

model.print_trainable_parameters()

==((====))==  Unsloth 2025.10.8: Fast Qwen3_Vl patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.72G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

trainable params: 51,346,944 || all params: 8,818,470,640 || trainable%: 0.5823


In [ ]:
from trl.trainer.sft_trainer import DataCollatorForVisionLanguageModeling
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(MODEL_ID)
collator = DataCollatorForVisionLanguageModeling(processor)

# 5. 프롬프트 템플릿 및 데이터 포맷팅

학습 및 추론에 사용할 프롬프트 템플릿을 정의하고, Hugging Face `datasets` 라이브러리를 사용하여 데이터를 모델의 Chat Template에 맞게 변환합니다. 이 과정은 기존의 Custom Dataset/Dataloader를 대체합니다.

In [ ]:
# Pandas DataFrame을 Hugging Face Dataset으로 변환
raw_dataset = Dataset.from_pandas(train_df)

# Resize to (512, 512) and handle image loading errors
def convert_to_rgb(example):
    try:
        example["decoded_image"] = Image.open(example["path"]).resize((512, 512)).convert("RGB")
    except Exception as e:
        print(f"Error loading image {example['path']}: {e}")
        # Create a white dummy image
        example["decoded_image"] = Image.new("RGB", (512, 512), color = 'white')
    return example

raw_dataset = raw_dataset.map(convert_to_rgb)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# 시스템 프롬포트도 커스터마이징 필수! (정확률 5~15% 향상)
SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

# 학습 데이터를 모델의 대화 형식(messages)으로 변환하는 함수
def make_conversation(example):
    user_text = build_mc_prompt(str(example["question"]), str(example["a"]), str(example["b"]), str(example["c"]), str(example["d"]))
    gold = str(example["answer"]).strip().lower()

    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {
            "role":"user",
            "content":[
                {"type":"image"}, # Placeholder for the image
                {"type":"text","text":user_text}  # The text part of the prompt
                ]
         },
          {
              "role":"assistant",
              "content":[
                  {"type":"text","text":gold}
                  ]
            }
    ]
    # The actual image data is kept separate for the processor
    return {"messages": messages, "images": [example["decoded_image"]]}

dataset = raw_dataset.map(make_conversation, remove_columns=["path", "a", "b", "c", "d", "question", "id", "answer", "decoded_image"])

formatted_dataset = dataset

# 훈련/검증 데이터 분리
dataset_split = formatted_dataset.train_test_split(test_size=0.1, seed=SEED)
train_dataset = dataset_split["train"]
valid_dataset = dataset_split["test"]

print("Final training sample:\n", train_dataset[0])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Final training sample:
 {'messages': [{'content': [{'text': None, 'type': 'image'}, {'text': '사진 속 새는 어떤 종류의 새일까요?\n(a) 앵무새\n(b) 까마귀\n(c) 비둘기\n(d) 참새\n\n정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.', 'type': 'text'}], 'role': 'user'}, {'content': [{'text': 'a', 'type': 'text'}], 'role': 'assistant'}], 'images': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512 at 0x7AA3526881D0>]}


In [ ]:
image = train_dataset[100]["images"]
text = train_dataset[100]["messages"]

print(image)
print("=================================================================")
print(text)

[<PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512 at 0x7AA35226D820>]
[{'content': [{'text': None, 'type': 'image'}, {'text': '이 사진에서 볼 수 있는 것은 무엇인가요?\n(a) 공항과 비행기\n(b) 도시의 전경과 시장\n(c) 해변과 바다\n(d) 산과 숲\n\n정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요.', 'type': 'text'}], 'role': 'user'}, {'content': [{'text': 'b', 'type': 'text'}], 'role': 'assistant'}]


# 6. SFTTrainer를 사용한 파인튜닝

Unsloth와 `trl` 라이브러리의 `SFTTrainer`를 사용하여 모델 파인튜닝을 진행합니다. 이 방식은 기존의 수동 학습 루프보다 훨씬 간결하고 효율적입니다.

공식 docs : https://huggingface.co/docs/trl/main/sft_trainer

- 200개 샘플 학습 시 약 5~10분 소요됩니다.

In [ ]:
# --- 3. 모델 학습 ---

# 모델을 학습 모드로 활성화 (기존과 동일)
FastVisionModel.for_training(model)

# SFTTrainer 설정 (기존과 동일)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # packing=True,           # 여러 짧은 시퀀스를 하나로 묶어 학습 효율을 높입니다.
    data_collator=collator,
    args=SFTConfig(
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=1e-4,
        logging_steps=1,
        eval_strategy="steps",
        eval_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=SEED,
        output_dir="outputs",
        report_to="none",
        remove_unused_columns=False,
        dataset_kwargs={"skip_prepare_dataset": True},
        max_length=MAX_SEQ_LENGTH,
    ),
)

# 학습 시작
print("Starting training...")
trainer.train()
print("Training finished.")

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 180 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 51,346,944 of 8,818,470,640 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,3.778100,14.385533
20,2.309900,8.594125
30,1.770700,7.129172
40,1.634100,6.449550
50,1.574000,6.214262
60,1.540600,6.159873


Unsloth: Not an error, but Qwen3VLForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


Training finished.


# 7. 추론 및 제출 파일 생성

학습된 LoRA 어댑터를 사용하여 테스트 데이터에 대한 추론을 수행하고, `submission.csv` 파일을 생성합니다.

- 전체 테스트 데이터 추론 시 약 30분~1시간 소요됩니다.

In [ ]:
FastVisionModel.for_inference(model) # 추론 모드 활성화

# 모델의 응답에서 정답(a, b, c, d)만 추출하는 함수
def extract_choice(text: str) -> str:
    text = text.strip().lower()
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines:
        return "a" # 응답이 없는 경우 'a'로 대체
    last = lines[-1]
    if last in ["a", "b", "c", "d"]:
        return last
    tokens = last.split()
    for tok in tokens:
        if tok in ["a", "b", "c", "d"]:
            return tok
    return "a" # 정답을 찾지 못한 경우 'a'로 대체

# 추론 루프
preds = []
for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]
    image = Image.open(row["path"]).convert("RGB")
    user_text = build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])

    # Updated prompt structure to correctly pass text
    messages = [
        {"role":"system","content":[{"type":"text","text":SYSTEM_INSTRUCT}]},
        {
            "role":"user",
            "content":[
                {"type":"image"}, # Placeholder for the image
                {"type":"text","text":user_text}  # The text part of the prompt
                ]
         }
      ]

    prompt = tokenizer.apply_chat_template(
            messages,
            tokenize = False,
            add_generation_prompt = True, # Must add assistant
    )

    # Chat Template 적용 - Pass the list of dictionaries directly to the tokenizer
    # Corrected tokenizer call to pass the text and image separately
    inputs = tokenizer(
        image,
        prompt,
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")


    outputs = model.generate(
        **inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        # use_cache=True
    )

    # skip_special_tokens=True로 특수 토큰 제거
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    preds.append({"id": row["id"], "answer": extract_choice(text)})

# 제출 파일 생성
submission = pd.DataFrame(preds, columns=['id', 'answer'])
submission.to_csv("/content/submission.csv", index=False)
print("Saved /content/submission.csv")
print("Submission file sample:")
print(submission.head())

In [ ]:
# 나의 구글 드라이브 본래 작업 폴더에 저장
drive_path = "/content/drive/My Drive/251024/submission.csv"
df.to_csv(drive_path, index=False)
print(f"Saved to Google Drive: {drive_path}")

In [ ]:
# lora 어뎁터 저장
# model.save_pretrained("ai_challenge_lora")  # Local saving
# tokenizer.save_pretrained("ai_challenge_lora")